In [1]:
import pandas as pd
import os
import numpy as np
from dateutil import parser
from datetime import datetime
import time

In [2]:
import re


def get_xlsx_data(directory):
    # 检查目录是否存在
    if not os.path.isdir(directory):
        print("提供的路径不是一个有效的目录")
        return

    # 初始化一个空的 DataFrame 来存储所有文件的数据
    all_data = pd.DataFrame()
    num_of_all_files = 0
    num_of_targeted_files = 0
    skip_files_list = []

    # 遍历目录中的所有 Excel 文件
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".xlsx"):
            num_of_all_files += 1
        
        if file_path.endswith('数据开放.xlsx'):
            print(f"获取 {filename} 成功")
            num_of_targeted_files += 1
            # 读取 Excel 文件
            df = pd.read_excel(file_path)
            if 'location' not in df.columns:
                # 增加省份名称
                parts = filename.split('_')
                df['location'] = parts[0]
                print(f'{filename} 缺少 location 列，已自动添加')
            # 添加数据到总 DataFrame
            all_data = pd.concat([all_data, df], ignore_index=True)
        elif filename.endswith('.xlsx'): 
            skip_files_list.append(filename)
    
    print(all_data.head())
    # 检查必要的列是否存在
    required_columns = ['location', 'title', 'subject', 'description', 'source_department',
                        'release_time', 'update_time', 'open_conditions', 'data_volume',
                        'is_api', 'file_type', 'access_count', 'download_count',
                        'api_call_count', 'link', 'update_cycle']
    for col in required_columns:
        if col not in all_data.columns:
            print(f"缺少 {col} 列")
            return
    
    print(f"共获取 {num_of_targeted_files} 个有效文件，共 {num_of_all_files} 个文件")
    print(f"跳过 {len(skip_files_list)} 个文件：{skip_files_list}")
    return all_data

def remove_chinese_punctuation(text):
    """去除字符串中的中文标点符号"""
    chinese_punctuation = r'[\u3000-\u303F\uFF00-\uFFEF]'
    cleaned_text = re.sub(chinese_punctuation, '', text)
    return cleaned_text


In [3]:

# 省份编码表
province_codes = {
    '北京市': 1, '天津市': 2, '上海市': 3, '重庆市': 4, '河北省': 5, '福建省': 6, '江西省': 7, '山东省': 8,
    '山西省': 9, '内蒙古自治区': 10, '河南省': 11, '辽宁省': 12, '湖北省': 13, '吉林省': 14, '湖南省': 15,
    '黑龙江省': 16, '广东省': 17, '江苏省': 18, '广西壮族自治区': 19, '海南省': 20, '四川省': 21, '浙江省': 22,
    '贵州省': 23, '安徽省': 24, '云南省': 25, '陕西省': 26, '西藏藏族自治区': 27, '甘肃省': 28, '青海省': 29,
    '宁夏回族自治区': 30, '新疆维吾尔自治区': 31
}

# 拼音到中文的映射
pinyin_to_chinese = {
    'Anhui': '安徽省', 'Beijing': '北京市', 'Chongqing': '重庆市',
    'Fujian': '福建省', 'Guangdong': '广东省', 'Guangxi': '广西壮族自治区',
    'Guizhou': '贵州省', 'Hainan': '海南省', 'Hebei': '河北省',
    'Hubei': '湖北省', 'Hunan': '湖南省', 'Jiangsu': '江苏省',
    'Jiangxi': '江西省', 'Liaoning': '辽宁省', 'Ningxia': '宁夏回族自治区',
    'Shandong': '山东省', 'Shanghai': '上海市', 'Shanxi': '山西省',
    'Sichuan': '四川省', 'Tianjin': '天津市', 'Zhejiang': '浙江省'
}

set_undefined = {}

open_condition_undefined = {}

# 建立更新周期与每年更新次数的映射关系
frequency_map = {
    '每年': 1/12, '按年': 1/12, '按年更新': 1/12,
    '每半年': 2/12, '半年更新': 2/12,
    '每两年': 1/(2*12), '2年': 1/(2*12),
    '每三年': 1/(3*12), '自定义：三年': 1/(3*12), '3年': 1/(3*12), '三年': 1/(3*12),
    '每五年': 1/(5*12), '每十年': 1/(10*12), '自定义：10年': 1/(10*12),
    '自定义：每季度': 4/12, '按季度': 4/12, '按季更新': 4/12, '每季度': 4/12, 
    '每月': 1, '按月': 1, '按月更新': 1,
    '每周': 52/12, '按周': 52/12, '按周更新': 52/12,
    '每日': 365/12, '按日更新': 365/12, '每天': 365/12, '按天': 365/12,
    '实时': 365/12, '即时': 365/12, '实时更新': 365/12,
    '按小时': 24 * 30, '小时级': 24 * 30,
    '按分钟': 60 * 24 * 30, '分钟级': 60 * 24 * 30,
    '不定期': 0, '不更新': 0, '静态数据': 0, '未定义': 0, '不定期更新': 0,
    '自定义': 0, '自定义：不更新': 0, '自定义：静态文件不更新': 0, 
    '自定义：数据截至2020年': 0, '自定义：不在此平台更新': 0,
    '自定义：不在此平台发布更新': 0, '自定义：历史数据不更新': 0, 
    '自定义：无需更新': 0, '自定义：无变化不更新': 0,
    '自定义：2年': 1/(2*12), '自定义：3年': 1/(3*12), '自定义：每三年': 1/(3*12), 
    '自定义：有变动时更新': 0, '自定义：不定期，有变动时更新': 0,
    '自定义：每两年': 1/(2*12), '自定义：按需更新': 0, 
    '自定义：每个评选周期': 0, '自定义：5年': 1/(5*12), '自定义：每年': 1/12, 
    '自定义：五年': 1/(5*12), '自定义：0': 0, '自定义：两年': 1/(2*12), 
    '自定义：每十年': 1/(10*12), '自定义：每5年': 1/(5*12),
    '自定义：有新许可单位时': 0, '自定义：开展新一轮评选后更新': 0, 
    '自定义：新增数据后更新': 0, '自定义：新增后更新': 0, '自定义：不定时更新': 0, 
    '自定义：产生新数据时更新': 0, '自定义：有新数据时更新': 0, '自定义：不定时': 0,
    '自定义：有更新的信息就更新': 0, '自定义：有新的类型再进行更新': 0, '自定义：用户自定义': 0,
    '自定义：有自然灾害时统计更新': 0, '自定义：人口普查工作开展时更': 0, 
    '自定义：每次普查结果实时更新': 0, '自定义：申请维修支出时查询': 0, 
    '自定义：有案件更新时更新数据': 0, '自定义：高院发布数据时更新': 0, 
    '自定义：有普查更新时更新': 0, '自定义：普查时更新': 0, '自定义：有更新时更新': 0, 
    '自定义：通邮建制村所属': 0, '自定义：发生实际变更时': 0, '自定义：按批复文进行更新': 0,
    '自定义：根据实际情况': 0, '自定义：以后由应急管理局发': 0, '自定义：以后由应急局发': 0, '自定义：不定期': 0, '自定义：有变化时更新': 0, '自定义：': 0, '自定义：不定期更新': 0,
    '自定义：数据有更新时更新': 0, '其他': 0, '其它': 0, 
    '每年 (更新说明:暂无新业务数据产生)': 1/12, '每月 (更新说明:暂无新业务数据产生)': 1,
    '每季度 (更新说明:暂无新业务数据产生)': 4/12, '不更新 (更新说明:暂无新业务数据产生)': 0,
    '不定期 (更新说明:暂无新业务数据产生)': 0, '每日 (更新说明:暂无新业务数据产生)': 365/12,
    '小时级 (更新说明:暂无新业务数据产生)': 24 * 30, '每周 (更新说明:暂无新业务数据产生)': 52/12,
    '分钟级 (更新说明:暂无新业务数据产生)': 60 * 24 * 30
}

# 将 'is_api' 字段转换为布尔值
def convert_to_bool(value):
    if value is None or value == '':
        return None
    
    if isinstance(value, str):
        return value.lower() == 'true'
    return bool(value)

def convert_data_volume(volume):
    """
    将不同格式的数据容量转换为统一的整数形式，单位为MB。

    参数:
    volume (str, float, or int): 原始数据容量字符串或数值，如 "1G", "0.5", "30M" 或数字。

    返回:
    int: 转换后的数据容量整数，若为 "若干" 则返回 1，无法转化为整数则返回 0。
    """
    if volume is None or volume == '':
        return None  # 保留为空
    try:
        if isinstance(volume, str):  # 如果volume是字符串，执行单位转换
            volume = volume.strip()  # 去除字符串两端的空格
            if volume == '若干':
                return 1  # 转换为1
            if 'G' in volume:
                return int(float(volume.upper().replace('G', '')) * 1024)  # 转换为MB，转为整数
            elif 'M' in volume:
                return int(float(volume.upper().replace('M', '')))  # 去除M，转换为整数
            else:
                return int(float(volume))  # 没有单位，转换为浮点数后转为整数
        elif isinstance(volume, (float, int)):  # 检查volume是否为数值（包括float和int）
            return int(volume)  # 转换为整数
    except ValueError as e:
        return None  # 返回0，避免在转换为整数时出现错误

    return None  # 返回0，避免没有匹配到任何情况时的错误

def calculate_update_frequency(row):
    """
    根据更新周期计算每月更新频率。

    参数:
    row (Series): 包含了需要的更新信息的行数据。

    返回:
    float: 每月更新次数。
    """
    update_cycle = row['update_cycle']
    # 若更新周期为空，则返回None
    if update_cycle is None or update_cycle == '' or pd.isna(update_cycle) :
        return None
    
    release_time = pd.to_datetime(row['release_time'], errors='coerce')
    update_time = pd.to_datetime(row['update_time'], errors='coerce')

    # 如果发布时间晚于更新时间，则不更新
    if release_time >= update_time:
        return None

    # 如果更新周期不是字符串，直接返回0
    if not isinstance(update_cycle, str):
        return None
    update_cycle = update_cycle.replace("‘", '').replace("’",'').replace('自定义：', '').strip()
 
    if update_cycle not in frequency_map:
        if update_cycle not in set_undefined:
            set_undefined[update_cycle] = 1
        else:
            set_undefined[update_cycle] += 1
        return 0

    # 若更新周期在映射字典中，则返回每月更新次数，否则返回0
    return frequency_map.get(update_cycle, None) / 12

def calculate_timed_update_rate(row):
    """
    根据更新周期计算每月更新频率。

    参数:
    row (Series): 包含了需要的更新信息的行数据。

    返回:
    float: 每月更新次数。
    """
    update_cycle = row['update_cycle']
    # 若更新周期为空，则返回None
    if update_cycle is None or update_cycle == '' or not isinstance(update_cycle, str):
        return None
    
    update_cycle = update_cycle.replace("‘", '').replace("’",'').replace('自定义：', '').strip()

    
    if update_cycle not in frequency_map:
        update_cycle = str(update_cycle)
        if update_cycle not in set_undefined:
            set_undefined[update_cycle] = 1
        else:
            set_undefined[update_cycle] += 1
        return 0

    # 若更新周期在映射字典中，则返回每月更新次数，否则返回0
    return frequency_map.get(update_cycle, 0)
    

def calculate_is_update(row):
    """
    计算更新频率。

    参数:
    row (Series): DataFrame中的一行。

    返回:
    bool: 指示文本是否有更新。
    """
    release_time = pd.to_datetime(row['release_time'], errors='coerce')
    update_time = pd.to_datetime(row['update_time'], errors='coerce')

    # 如果发布时间晚于更新时间，则不更新
    if pd.notnull(release_time) and pd.notnull(update_time) and release_time < update_time:
        return True
    elif pd.isnull(release_time) or pd.isnull(update_time):
        return None
    else:
        return False
    
def map_open_conditions(value):
    if pd.isnull(value) or value == '' or value == 'nan':
        return None
    
    if isinstance(value, str):
        value = remove_chinese_punctuation(value)
    
    if value not in open_conditions_mapping:
        if value not in open_condition_undefined:
            open_condition_undefined[value] = 1
        else:
            open_condition_undefined[value] += 1
        return value
    
    return open_conditions_mapping[value]

def safe_parse_datetime(date_str):
    """尝试解析日期字符串，兼容多种格式，如果失败则返回 pd.NaT."""
    try:
        # dateutil.parser.parse 更加灵活，可以解析不完全一致的日期格式
        return parser.parse(date_str)
    except ValueError:
        return pd.NaT


In [4]:
# 使用示例
folder_path = r'../../docs/after_provinces_output_files'
output_path = r'../../docs'
df = get_xlsx_data(folder_path)

获取 上海市_数据开放.xlsx 成功
获取 北京市_数据开放.xlsx 成功
获取 四川省_数据开放.xlsx 成功
获取 天津市_数据开放.xlsx 成功
获取 宁夏回族自治区_数据开放.xlsx 成功
获取 安徽省_数据开放.xlsx 成功
获取 山东省_数据开放.xlsx 成功
获取 山西省_数据开放.xlsx 成功
山西省_数据开放.xlsx 缺少 location 列，已自动添加
获取 广东省_数据开放.xlsx 成功
获取 广西壮族自治区_数据开放.xlsx 成功
获取 江苏省_数据开放.xlsx 成功
获取 江西省_数据开放.xlsx 成功


C:\Users\dass1\AppData\Local\Temp\ipykernel_39328\283238742.py:33: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  all_data = pd.concat([all_data, df], ignore_index=True)


获取 河北省_数据开放.xlsx 成功
获取 浙江省_数据开放.xlsx 成功
获取 海南省_数据开放.xlsx 成功
获取 湖北省_数据开放.xlsx 成功
湖北省_数据开放.xlsx 缺少 location 列，已自动添加
获取 湖南省_数据开放.xlsx 成功
获取 福建省_数据开放.xlsx 成功
获取 贵州省_数据开放.xlsx 成功
获取 辽宁省_数据开放.xlsx 成功
获取 重庆市_数据开放.xlsx 成功
  location                      title subject  \
0      上海市         1978年以来住宅投资和竣工建筑面积    城市建设   
1      上海市                上海市普通高等学校信息    教育科技   
2      上海市   上海旅游统计数据-本市接待入境旅游者人数（人次）    文化休闲   
3      上海市  上海旅游统计数据-旅游饭店客房平均出租率和平均房价    文化休闲   
4      上海市  上海旅游统计数据-入境外国人按主要客源国分（人次）    文化休闲   

                                         description source_department  \
0                               1978年以来上海住宅投资和竣工建筑面积            上海市统计局   
1       上海市高校专业设置情况及对外联系方式，包括学校名称、联系电话、学院名称、专业名称等内容。          上海市教育委员会   
2             统计本市接待入境旅游者人数（人次），包括月度人次，累计年度人次，增长率等数据         上海市文化和旅游局   
3  旅游饭店客房平均出租率和平均房价，按照星级饭店和非星级饭店统计客房平均出租率和平均房价，并统...         上海市文化和旅游局   
4  入境外国人按主要客源国分（人次），按照主要客源国国别统计入境外国人过夜人数，按月统计，并计算增长率         上海市文化和旅游局   

          release_time          update_time op

In [5]:
# # 获取 update_cycle 列的所有唯一值
# unique_update_cycle = df['update_cycle'].unique()
# 
# unique_update_cycle

In [6]:
df_copy = df.copy()

# df_copy['location'] = df_copy['location'].map(pinyin_to_chinese)
    
df_copy['data_volume'] = df_copy['data_volume'].apply(convert_data_volume)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 数据容量转换完成")

# 计算平均每月更新频率
# df_copy['平均更新/月'] = df_copy.apply(calculate_update_frequency, axis=1).round(2)
# print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 平均更新频率计算完成")
# print("以下是未知的更新周期：", set_undefined)

# 计算平均每月更新频率
df_copy['定时更新率'] = df_copy.apply(calculate_timed_update_rate, axis=1).round(2)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 定时更新率计算完成")
print("以下是未知的更新周期：", set_undefined)

# 计算是否更新列
df_copy['is_update'] = df_copy.apply(calculate_is_update, axis=1)

# 确保release_time和update_time列是字符串类型
df_copy['release_time'] = df_copy['release_time'].astype(str)
df_copy['update_time'] = df_copy['update_time'].astype(str)

# 将 "nan" 替换为空字符串
df_copy['chosen_time'] = df_copy['release_time'].fillna('').replace('nan', '')
# 使用numpy.where选择release_time或update_time
# 将 release_time 为空字符串或者为字符串 "nan" 的情况下，使用 update_time 来填充 chosen_time
df_copy['chosen_time'] = np.where(df_copy['chosen_time'] == '', df_copy['update_time'], df_copy['chosen_time'])

# 应用自定义的安全日期解析函数，并提取年份
df_copy['year'] = df_copy['chosen_time'].apply(safe_parse_datetime).dt.year

 # 将x映射为相应的开放条件
open_conditions_mapping = {
'无条件开放': '无条件开放', '普通公开,简单注册用户可下载': '无条件开放',
'无条件': '无条件开放', '无条件更新': '无条件开放',
'登录开放': '无条件开放',
'普遍开放': '无条件开放',
'实名认证开放': '无条件开放',
'普通开放': '无条件开放',
'有条件': '有条件开放',
'有条件开放': '有条件开放',
'依申请开放': '有条件开放',
'审核开放': '有条件开放',
'有条件共享': '有条件开放',
'实名注册用户可下载': '有条件开放',
'申请开放': '有条件开放',
'完全公开': '无条件开放',
'部门审批': '有条件开放',
'审批开放': '有条件开放',
'依申请': '有条件开放',
'依条件开放': '有条件开放',
'根据业务需求提供': '有条件开放',
'依申请公开': '有条件开放',
'已申请开放': '有条件开放',
'以申请开放': '有条件开放',
'申请': '有条件开放',
'需要用户授权': '有条件开放',
'完全开放': '无条件开放',
'半开放': '有条件开放',
'不开放': '无条件开放',
'开放': '无条件开放',
'参考': '无条件开放',
'需申请使用': '有条件开放',
'依具体申请原因审核是否予以开放': '有条件开放',
'脱敏开放': '有条件开放',
'平台审核': '有条件开放',
'申请审批': '有条件开放',
'平台审批': '有条件开放',
'需申请': '有条件开放',
'工作参考': '无条件开放',
'无条件开发': '无条件开放',
'工作所需': '无条件开放',
'申请后公开': '有条件开放',
'申请后开放': '有条件开放',
'无条件公开': '无条件开放',
'用于数据核验': '无条件开放',
'五条件共享': '无条件开放',  # 有疑问，请确认
'无条件开房': '无条件开放',  # 有疑问，请确认
'申请公开': '有条件开放',
'需要提供查询函': '有条件开放',
'行政依据': '无条件开放',
'无条件共享，工作参考': '无条件开放',
'须提供具体申请依据、实际用途和申请主体的相关信息。': '有条件开放',
'眉山市有效外观明细表': '无条件开放',  # 有疑问，请确认
'开放数据': '无条件开放',
'社会保障卡启用': '无条件开放',  # 有疑问，请确认
'办理施工许可证证可以申请查看': '有条件开放',
'安全监督备案表': '无条件开放',
'无条件开共享': '无条件开放',
'此项政务信息资源仅用于数据校核、业务协同。': '无条件开放',
'此数据仅限于工作参考和数据校核': '无条件开放',
'本数据仅限用于工作参考': '无条件开放',
'办公人员联络方式用于业务协调': '无条件开放',
'可共享': '无条件开放',
'无条件开放。': '无条件开放',
'市级部门无条件共享': '无条件开放',
'无条件共共享': '无条件开放',  # 有疑问，请确认
'眉山市路灯照明工程情况': '无条件开放',  # 有疑问，请确认
'有条件工程': '有条件开放',  # 有疑问，请确认
'有': '有条件开放',
'仅作工作参考': '无条件开放',
'数据仅用于业务办理查询': '有条件开放',
'数据仅用于业务办理': '有条件开放',
'数据仅用于业务办理和查询': '有条件开放',
'数据用于业务办理': '有条件开放',
'不予共享': '无条件开放',
'业务协同': '无条件开放',
'WU': '无条件开放',  # 有疑问，请确认
'仅用于工作参考': '无条件开放',
'数据仅用于共享业务查询': '有条件开放',
'农村计划生育家庭奖励扶助情况': '无条件开放',  # 有疑问，请确认
'独生子女父母专项奖励情况': '无条件开放',  # 有疑问，请确认
'共享': '无条件开放',
'用作工作参考': '无条件开放',
'依据：《中华人民共和国森林法》和《森林法实施条例》。': '无条件开放',  # 有疑问，请确认
'提供查询的函。': '有条件开放',
'用作业务协同': '无条件开放',
'仅对上级法院系统开放': '有条件开放',
'无条件g共享': '无条件开放',  # 有疑问，请确认
'依申请共享': '有条件开放',
'经申请': '有条件开放',
'该数据仅供参考，如有疑问请联系眉山市公安局天府新区分局。': '有条件开放',  # 有疑问，请确认
'公开数据': '无条件开放',
'有条件开发': '有条件开放',
'对所有人员公开': '无条件开放',
'已申请': '有条件开放',
'向社会开放': '无条件开放',
'无条件开放共享': '无条件开放',
'身份证申请或申请书': '有条件开放',
'眉山天府新区林区分布一览表': '无条件开放',  # 有疑问，请确认
'彭山区科普惠民共享基地名单': '无条件开放',  # 有疑问，请确认
'区科协开展科技下乡活动统计': '无条件开放',  # 有疑问，请确认
'彭山区农技协名单': '无条件开放',  # 有疑问，请确认
'先申请': '有条件开放',
'无条件共享类': '无条件开放',  # 有疑问，请确认
'与残联相关业务': '无条件开放',  # 有疑问，请确认
'无。': '无条件开放',  # 有疑问，请确认
'青神县融媒体中心广播电视编辑记者采编考试合格证持证情况': '无条件开放',  # 有疑问，请确认
'单位来函': '有条件开放',
'单位发函': '有条件开放',
'仅用于数据参考。': '无条件开放',
'仅用于数据参考': '无条件开放',
'仅用于工作参考。': '无条件开放',
'仅作为工作参考': '无条件开放',
'仅供工作参考': '无条件开放',
'仅用于工作参考、数据校核。': '无条件开放',
'仅用于工作参考，数据校核。': '无条件开放',
'用于数据校核、工作参考、业务协同。': '无条件开放',
'用于工作参考可申请共享': '有条件开放',
'用于工作需要可申请共享': '有条件开放',
'洪雅县外贸企业名单': '无条件开放',  # 有疑问，请确认
'作为工作参考。': '无条件开放',
'作为行政执法依据进行公开': '无条件开放',
'行政公开': '无条件开放',
'申请同意后开放': '有条件开放',
'无条件分享': '无条件开放',
'有条件分享': '有条件开放',
'文件': '无条件开放',
'无条件开放数据': '无条件开放',
'数据仅用于业务查询办理': '有条件开放',
'向社会公开': '无条件开放',
'对所有人公开': '无条件开放',
'审计法治工作报告': '无条件开开放',
'无条件是使用，用于开展正常工作': '无条件开放',
'仅用于工作参考，业务协同。': '无条件开放',
'洪雅县地方政府债务限额汇总表': '无条件开放',  # 有疑问，请确认
'公务用车购置及运行费统计表': '无条件开放',  # 有疑问，请确认
'有条件公共享': '有条件开放',  # 有疑问，请确认
'彭山称号企业汇总': '无条件开放',  # 有疑问，请确认
'无条件共享，作为政务信息公开。': '无条件开放',
'作为行政依据、业务参考': '无条件开放',
'眉山市市级示范社名录': '无条件开放',  # 有疑问，请确认
'政务信息主动公开流程图': '无条件开放',  # 有疑问，请确认
'个体工商户变更登记流程图': '无条件开放',  # 有疑问，请确认
'个体工商户开业登记流程图': '无条件开放',  # 有疑问，请确认
'个体工商户注销登记流程图': '无条件开放',  # 有疑问，请确认
'名称争议事项流程图': '无条件开放',  # 有疑问，请确认
'有业务需求的进行共享': '有条件开放',
'信访条例': '无条件开放',
'预算法': '无条件开放',
'无条件贡献': '无条件开放',
'作为行政依据，工作参考。': '无条件开放',
'龙马镇商贸行业燃气安全隐患管理台账': '无条件开放',  # 有疑问，请确认
'依条件申请': '有条件开放',
'重点服务业企业培育情况表': '无条件开放',  # 有疑问，请确认
'需向部门申请': '有条件开放',
'政务信息资源': '无条件开放',
'区科协科普e展点位': '无条件开放',  # 有疑问，请确认
'彭山区农技协开展培训情况': '无条件开放',  # 有疑问，请确认
'需要可自行下载': '无条件开放',
'开发共享下载': '无条件开放',
'开发下载': '无条件开放',
'开放下载': '无条件开放',
'可作为工作参考': '无条件开放',
'需求部门申请共享': '有条件开放',
'无条件分析': '无条件开放',
'廉政法规知识测试资料': '无条件开放',
'丹棱县村级小微权力清单': '无条件开放',  # 有疑问，请确认
'丹棱县纪委年度部门预算': '无条件开放',  # 有疑问，请确认
'群众信访举报途径电子表格数据': '无条件开放',  # 有疑问，请确认
'用于档案数据校验': '无条件开放',
'地方政府专项债券分区域限额': '无条件开放',  # 有疑问，请确认
'用于档案查阅利用': '无条件开放',
'申请同意后开放共享': '有条件开放',
'眉山市公共资源交易领域信息主动公开目录': '无条件开放',  # 有疑问，请确认
'需要可以出函申请': '有条件开放',
'共享平台': '无条件开放',
'需个人或单位出具查档介绍信或查档函': '有条件开放',
'获奖名单': '无条件开放',  # 有疑问，请确认
'招生依据': '无条件开放',  # 有疑问，请确认
'用于数据校核': '无条件开放',
'非盈利性使用': '无条件开放',
'无条件开发1': '无条件开放',  # 有疑问，请确认
'查询人基本信息，身份证信申请查询': '有条件开放',
'待县人大批准2020年部门决算后开放': '有条件开放',
'县人大批准2020年部门决算后开放': '有条件开放',
'无条件共享开放': '无条件开放',
'眉山司法局香港、澳门永久性居民中的中国居民申请在眉从事律师职业核准办事指南': '无条件开放',  # 有疑问，请确认
'中华人民共和国公证法': '无条件开放',
'眉山市司法局兼职律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局律师事务所设立办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局关于设立律师事务所（分所）的指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局法律援助律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局台湾居民申请在川从事律师职业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山市司法局专职律师执业许可办事指南': '无条件开放',  # 有疑问，请确认
'眉山公证机构及公证员基本信息': '无条件开放',  # 有疑问，请确认
'共享类文件': '无条件开放',
'农村居民可支配收入与支出': '无条件开放',  # 有疑问，请确认
'2020年丹棱县本级一般公共预算支出预算表': '无条件开放',  # 有疑问，请确认
'2020年丹棱县政府性基金收入预算表': '无条件开放',  # 有疑问，请确认
'填写申请后开放': '有条件开放',
'有条件共享（申请）': '有条件开放',
'登录公开': '无条件开放',
'按相关规定审核开放共享': '有条件开放',
'按需审批使用': '有条件开放',
'按需审核使用': '有条件开放',
'用于数据校验': '无条件开放',
'垃圾分类等场景': '无条件开放',  # 有疑问，请确认
'按相关政策要求，当前数据允许开放': '无条件开放',
'招聘会信息，无条件开放': '无条件开放',
'无条件开': '无条件开放',
'受限开放': '有条件开放',
'用于数据查看与应用': '无条件开放',
'因城市管理、综合执法需要可使用本数据': '无条件开放',
'按需开放': '有条件开放',
'开放网站': '无条件开放',
'申请共享': '有条件开放',
'开放网站申请': '有条件开放',
'根据接口规范': '无条件开放',
'1级\t不脱敏': '无条件开放',  # 有疑问，请确认
'依据申请': '有条件开放',
    '无': '无条件开放', '无条件共享': '无条件开放', '普通公开简单注册用户可下载': '无条件开放', 
}
df_copy['open_conditions'] = df_copy['open_conditions'].apply(map_open_conditions)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - open_conditions 列已转换为开放条件")
print("以下是无法映射的open_conditions:", open_condition_undefined)

df_copy['is_api'] = df_copy['is_api'].apply(convert_to_bool)
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - is_api 列已转换为布尔值")

df_copy['subject'] = df_copy['subject'].fillna('无分类主题')
df_copy['subject'] = df_copy['subject'].str.replace(" ", "")
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 'subject' 列空值填充完成")

df_copy['source_department'] = df_copy['source_department'].fillna('无来源部门')
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} -'source_department' 列空值填充完成")

df_copy['title'] = df_copy['subject'].fillna('无标题')
print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 'title' 列空值填充完成")

# 添加省份编号列
df_copy['省份编号'] = df_copy['location'].map(province_codes)

# 将chosen_time列转换为日期时间格式
df_copy['chosen_time'] = pd.to_datetime(df_copy['chosen_time'], errors='coerce')
# 分组并找到每个分组内chosen_time最早的值
df_copy['数据初次发布时间'] = df_copy.groupby('location')['chosen_time'].transform('min').dt.strftime('%Y-%m-%d')

2025-03-06 15:31:34 - 数据容量转换完成
2025-03-06 15:31:38 - 定时更新率计算完成
以下是未知的更新周期： {'用户自定义': 63, '有变动时更新': 31, '不定期，有变动时更新': 1, '数据截至2020年': 1, '有变化时更新': 9, '按需更新': 14, '工业总量不对外': 1, '投资总量数据不对外': 1, '': 7, '10年': 1, '静态文件不更新': 2, '每个评选周期': 2, '5年': 2, '五年': 4, '0': 10, '不在此平台更新': 1, '不在此平台发布更新': 3, '两年': 1, '有新许可单位时': 1, '开展新一轮评选后更新': 1, '新增数据后更新': 1, '新增后更新': 1, '不定时更新': 7, '产生新数据时更新': 13, '无需更新': 4, '有新数据时更新': 2, '不定时': 5, '每5年': 2, '历史数据不更新': 5, '有更新的信息就更新': 1, '无变化不更新': 2, '有新的类型再进行更新': 1, '有自然灾害时统计更新': 1, '人口普查工作开展时更': 1, '每次普查结果实时更新': 1, '申请维修支出时查询': 1, '有案件更新时更新数据': 1, '高院发布数据时更新': 1, '有普查更新时更新': 1, '普查时更新': 1, '有更新时更新': 1, '通邮建制村所属': 1, '发生实际变更时': 1, '按批复文进行更新': 1, '根据实际情况': 2, '以后由应急管理局发': 1, '以后由应急局发': 1, '数据有更新时更新': 1, '实时 (更新说明:暂无新业务数据产生)': 1}
2025-03-06 15:32:44 - open_conditions 列已转换为开放条件
以下是无法映射的open_conditions: {}
2025-03-06 15:32:44 - is_api 列已转换为布尔值
2025-03-06 15:32:44 - 'subject' 列空值填充完成
2025-03-06 15:32:44 -'source_department' 列空值填充完成
2025-03-06 15:32:44 - 'title' 列空值填充完成


In [ ]:
df_copy

In [8]:
import pandas as pd

def safe_sum(x):
    """
    计算非空值的总和
    :param x: 包含数值的列表或Series
    :return: 非空值的总和，如果所有值都为空，则返回None
    """
    valid_values = [volume for volume in x if pd.notna(volume)]
    return sum(valid_values) if valid_values else None

def safe_mean(x):
    """
    计算非空值的平均值
    :param x: 包含数值的列表或Series
    :return: 非空值的平均值，如果所有值都为空，则返回None
    """
    valid_values = [value for value in x if pd.notna(value)]
    return sum(valid_values) / len(valid_values) if valid_values else None

def safe_condition_mean(condition):
    """
    计算满足条件的非空值的比例
    :param condition: 条件值
    :return: 一个函数，该函数计算满足条件的非空值的比例，如果所有值都为空，则返回None
    """
    def inner(x):
        valid_values = [value for value in x if pd.notna(value)]
        count_condition = sum(1 for value in valid_values if value == condition)
        return count_condition / len(valid_values) if valid_values else None
    return inner

def proportion_greater_than_zero(series):
    if series.isna().all():
        return None
    
    return (series > 0).mean()

In [9]:
# 确保数据帧按照分组键排序
df_result = df_copy.sort_values(by=['location', 'source_department', 'subject', 'year'])

# 设置分组键为索引
df_result = df_result.set_index(['location', 'source_department', 'subject', 'year'])

# 分组统计各项数据
grouped = df_result.groupby(level=[0, 1, 2, 3])


In [11]:

summary = grouped.agg({
    'title': 'count',  # 文本数
    'data_volume': safe_sum,  # 总数据量
    '定时更新率': proportion_greater_than_zero,  # 大于0的值的占比
    'is_update': safe_mean,  # 更新文本率
    'open_conditions': [
        safe_condition_mean('无条件开放'),  # 无条件开放文本率
        safe_condition_mean('有条件开放')  # 有条件开放文本率
    ],
    'is_api': safe_mean,  # 有接口文本率
    'access_count': safe_sum,  # 总访问次数
    'download_count': safe_sum,  # 总下载次数
    '省份编号': 'first',  # 省份编号
    '数据初次发布时间': 'first'  # 数据初次发布时间
}).reset_index()

# 重命名统计列
summary.columns = ['地区', '来源部门', '主题', '年份', '文本数', '总数据量', '定时更新率', '更新文本率', '无条件开放文本率', '有条件开放文本率', '有接口文本率', '总访问次数', '总下载次数', '省份编号', '该地区数据初次发布时间']

# 重新排列列顺序
new_column_order = ['地区', '省份编号', '该地区数据初次发布时间', '来源部门', '主题', '年份', '文本数', '总数据量', '定时更新率', '更新文本率', '无条件开放文本率', '有条件开放文本率', '有接口文本率', '总访问次数', '总下载次数']
summary = summary.reindex(columns=new_column_order)

# 保存统计信息到Excel文件
summary.to_excel(f'{output_path}/省级统计信息汇总_{time.strftime("%Y%m%d_%H%M%S")}.xlsx', index=False)

print("统计信息已保存至:", f'{output_path}/省级统计信息汇总_{time.strftime("%Y%m%d_%H%M%S")}.xlsx')


C:\Users\dass1\AppData\Local\Temp\ipykernel_39328\1222939555.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()
C:\Users\dass1\AppData\Local\Temp\ipykernel_39328\1222939555.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()
C:\Users\dass1\AppData\Local\Temp\ipykernel_39328\1222939555.py:15: PerformanceWarning: indexing past lexsort depth may impact performance.
  }).reset_index()


统计信息已保存至: ../../docs/省级统计信息汇总_20250306_153319.xlsx


In [12]:
df_copy.to_csv('./test.csv')